### Building an Excel File which includes all the UK Government Access Fuel Price Data

**This notebook has been built in the Anaconda enviroment, using Anaconda Navigtor 2.5.2 and Jupyter Notebook 7.02.**

**It using the dependencies already carried by Anaconda environment, such as BeautifulSoup**

Replicating this outside of Anaconda or different versions may require downloading dependent modules

In [2]:
#CODE TO BUILD EXCEL FILE in a .xlsx format containing all the data listed for fuel prices provided at the
# UK government ACCESS FUEL PRICES website: https://www.gov.uk/guidance/access-fuel-price-data

#import the modules we need to build and export the DataFrame

from bs4 import BeautifulSoup #beautifulsoup 4 package is installed as part of Anaconda environment to parse HTML
import requests 
import pandas as pd
import json
from datetime import datetime
import os  # this will allow us to specify which path we want to save the final output excel file to


#### Scraping the URLs we need from the UK government website: 

In [3]:
# SCRAPE THE URLs:

# Define the URL of the webpage to scrape
url = 'https://www.gov.uk/guidance/access-fuel-price-data'

# use requests library to send GET request for the url

response = requests.get(url)

# Check if the request was successful - status should be 200 and parse with BeautifulSoup

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

# Initialize a list to store JSON URLs
json_url = []


# The URLs are all held within table markdown tags. To find all <td> elements in the table, using soup:

td_elements = soup.find_all('td')

# Iterate over each <td> element and extract the URLs ending with .json
for td in td_elements:
    url = td.text.strip()
    if url.startswith('https://') and url.endswith('.json') or url.endswith('.html'):
        json_url.append(url)

#### Building the intitial DataFrame

In [4]:

#building the dataframe

#headers - WITHOUT THIS - THE tesco_url ENTRY WILL REJECT THE REQUEST AND TIME OUT

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

#blank list to store the called json data

fuel_data = []

#loop to iterate through the urls in json_url and then add them using .append to fuel_data

for url in json_url:
    url_data = requests.get(url, headers=headers)  #use the requests.get method and tells requests, and make sure headers are used 
    data = url_data.json()   
    fuel_data.append(data)   

# Make DataFrame with pandas (pd) called df_fuel - and use the .json_normalize function get to record_path 'stations'

df_fuel = pd.json_normalize(fuel_data, record_path = 'stations')

In [5]:
# returns the dataframe - for visual check - remove comment if required

# df_fuel


#### Filtering our DataFrame for Wales only postcode locations

The initial filter relies on using the two letter codes used in Wales - though some, such as CH, LD and HR also cover some England postcodes areas. This will cut the entries down substantially - allowing us to make use of an API to further check the postcodes locations

In [6]:

# Filter the DataFrame to include only postcodes starting with 'CF' or 'LL'
# This will cut the new dataframe from 4554 entries down to under 300

df_poss_wales = df_fuel[df_fuel['postcode'].str.startswith(('LL', 'NP', 'CF', 'SA', 'SY', 'CH', 'LD', 'HR3', 'HR5'))]



#### Checking our filtered list against a database - using an API call to postcodes.io database

**We first need to build a function that will allow us to make the api calls**

In [7]:
#We can now check to see which other postcodes are actually Wales postcodes - calling the 300 through the postcodes api
# We build this function first - which calls the api.postcodes url - and then checks the json file and nested entry 'country'
# if there's a match for Wales - it returns a True value - otherwise - it returns False

# Function to check if a postcode is in Wales
def is_postcode_in_wales(postcode):
    url = f"http://api.postcodes.io/postcodes/{postcode}"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()['result']
        if result and result['country'] == 'Wales':
            return True
    return False


#### Building an updated dataframe with a new columns 'Wales' which will list whether the postcode is in Wales (True) or not (False)

In [8]:
# Update the 'Wales' column based on the postcode check 
# this applies the function we've just built - adds a new column 'Wales' to the dataframe df_poss_wales - which will be either True or False



df_wales = df_poss_wales

df_wales['Wales'] = df_wales['postcode'].apply(is_postcode_in_wales)

/var/folders/96/y52ym7t12ss382mr_dtb__g00000gn/T/ipykernel_6161/816111262.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wales['Wales'] = df_wales['postcode'].apply(is_postcode_in_wales)


In [9]:
df_wales

,site_id,brand,address,postcode,location.latitude,location.longitude,prices.B7,prices.E10,prices.E5,prices.SDV,Wales
15,gcmudtu75hhr,AppleGreen,"Salop Road, Oswestry",SY11 2RL,52.854389,-3.048228,147.8,143.8,158.8,NaN,False
17,gcmg8dcg0n1p,AppleGreen,"Salop Road, Welshpool",SY21 7ET,52.662139,-3.139685,147.8,140.8,NaN,NaN,True
19,gcq4bhpx5gp9,AppleGreen,"Crossways, Church Stretton",SY6 6PQ,52.537839,-2.802029,150.8,145.8,160.8,NaN,False
24,gcjmnb068hbn,AppleGreen,"Tremains Road, Bridgend",CF31 1TZ,51.504362,-3.570213,143.8,140.8,NaN,NaN,True
29,gcjw75mdkp2p,AppleGreen,"3 - 3a Lower High Street, Merthyr Tydfil",CF47 8EB,51.741912,-3.376064,151.8,145.8,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
4484,gcjy29583mzz,TESCO,Tesco Stores Ltd Castle Street Abertillery,NP13 1UR,51.729201,-3.137219,149.9,144.9,151.9,NaN,True
4501,gcjn5ypv83ct,TESCO,Tidal Reach Pontarddulais Abertawe,SA4 8TA,51.713601,-4.043299,146.9,142.9,150.9,NaN,True
4504,gchqjq81juys,TESCO,Havens Head Park The Docks Milford Haven,SA73 3AU,51.715591,-5.042469,144.9,138.9,NaN,NaN,True
4515,gcm95nfc51ex,TESCO,Tesco Waterloo Road Llandrindod,LD1 6BH,52.244291,-3.379879,149.9,144.9,153.9,NaN,True


**While a warning is thrown up - it does actually create the correct dataframe, listing a column of True and False values for Wales**

#### We can now filter the new dataframe to remove all False entries

In [10]:
#removes all false enteries - leaving only Wales enteries

df_wales = df_wales[df_wales.Wales != False]

In [11]:
# THIS ALLOWS US TO SCROLL THROUGH THE ENTIRE DATA FRAME - it's only 300 rows

pd.set_option('display.max_rows', None)

In [14]:
df_wales


,site_id,brand,address,postcode,location.latitude,location.longitude,prices.B7,prices.E10,prices.E5,prices.SDV,Wales
17,gcmg8dcg0n1p,AppleGreen,"Salop Road, Welshpool",SY21 7ET,52.662139,-3.139685,147.8,140.8,NaN,NaN,True
24,gcjmnb068hbn,AppleGreen,"Tremains Road, Bridgend",CF31 1TZ,51.504362,-3.570213,143.8,140.8,NaN,NaN,True
29,gcjw75mdkp2p,AppleGreen,"3 - 3a Lower High Street, Merthyr Tydfil",CF47 8EB,51.741912,-3.376064,151.8,145.8,NaN,NaN,True
40,gcjjyucqgp8d,AppleGreen,"Neath Road, Swansea",SA6 8EF,51.663034,-3.920273,146.8,140.8,155.8,NaN,True
41,gcjjv3exz521,AppleGreen,"1093 Carmarthen Road, Swansea",SA5 4AN,51.64533,-3.98293,NaN,140.8,155.8,NaN,True
42,gcjjxj3ucqjf,AppleGreen,"Fabian Way, Swansea",SA1 8PA,51.621484,-3.908668,147.8,NaN,NaN,162.8,True
43,gcjm3zzvxzsn,AppleGreen,"Port Talbot Road, Swansea",SA13 1HN,51.591409,-3.779302,146.8,141.8,NaN,NaN,True
54,gcmy3c6d7svb,AppleGreen,"Wrexham Road, Mold",CH7 4HL,53.137161,-3.083567,149.8,144.8,NaN,NaN,True
63,gcmy2x6kjhm3,AppleGreen,"Roundabout Service Station King Street, Clwyd",CH7 1LB,53.170418,-3.138778,149.8,144.8,NaN,NaN,True
96,gcjyg47qu66q,BP,"Brecon Road, Brecon Road, Abergavenny.",NP7 7RB,51.825054,-3.027577,147.9,141.9,NaN,NaN,True


In [15]:
#show length of dataframe in rows

len(df_wales)

235

#### Data Validation and cleaning

In [12]:
#From looking at the data - it's clear duplicates exist - especially for Morrisons brands under both Morrisons and MFG
# we use the .duplicated method in Pandas - stating which column to look at - in our case 'postcode'
# The keep = False retains both sets of duplicates - so they can be checked physically to ensure they are true duplicates

duplicates_wales = df_wales[df_wales.duplicated(['postcode'], keep = False)]

In [13]:
# displays the dupicates - notice the two bottom are not a true duplicate - rather two petrol stations almost next to each other

duplicates_wales

,site_id,brand,address,postcode,location.latitude,location.longitude,prices.B7,prices.E10,prices.E5,prices.SDV,Wales
263,gcm8dw51gpm2,ASDA,"Station Road, Builth Wells",LD2 3SS,52.152438,-3.401506,149.7,145.7,NaN,NaN,True
1379,gcmx2dhz0g81,Morrisons,Marsh Road,LL18 2DF,53.317871,-3.487097,149.7,145.7,155.9,NaN,True
1416,gchyvynu3ne0,Morrisons,Parc Pensarn,SA31 2NF,51.845247,-4.30831,145.7,139.7,155.9,NaN,True
1425,gcjsxw3ztvtq,Morrisons,International Sports Village,CF11 0JP,51.451666,-3.183378,146.7,142.7,155.9,NaN,True
1437,gcjvg7d7bx28,Morrisons,Grange Road,NP44 1QP,51.655651,-3.018128,147.7,141.7,155.9,NaN,True
1472,gcjsmt660upu,Morrisons,Penny Way,CF63 4BA,51.401275,-3.270793,148.7,143.7,155.9,NaN,True
1488,gcjwwswdgrft,Morrisons,Beaufort Road,NP23 5WS,51.792812,-3.220911,147.7,142.7,NaN,NaN,True
1489,gcjm962ms3dr,Morrisons,Christchurch Road,SA12 7DA,51.60511,-3.811727,145.7,140.7,NaN,NaN,True
1500,gcmvg8328xz2,Morrisons,Ruthin Road,LL13 7TU,53.043493,-3.008506,144.7,138.7,155.9,NaN,True
1511,gcmyt0xj5rzf,Morrisons,High Street,CH4 8RU,53.177475,-2.934573,146.7,141.7,155.9,NaN,True


In [14]:
# amended to use site_id reference - see note below

# uses the .drop method to remove multiple index items - these are the duplicate index numbers

#df_wales_cleaner = df_wales.drop(index =[1379, 1416, 1425,1437,1472, 1488,1489,1500, 1511])

Instead of using .drop and index reference - we will adopt dropping the row by using the site_id reference, which remains the same - while the index may change, depending on how many stations overall are reported in a week.

We will used the format:
```
# Define the strings to remove
strings_to_remove = ['site_id_1', 'site_id2']

# Remove rows where 'Column_Name' contains any of the strings in the list
df = df[~df['Column_Name'].str.contains('|'.join(strings_to_remove))]

```

In [24]:
# Define strings to remove - using first duplicate values in duplicate wales (but not the first one - it's not a real duplicate
# rather it is a second petrol forecourt in the same postcode area - different brands)

strings_to_remove = ['gcm8dw51gpm2', 'gcmx2dhz0g81', 'gchyvynu3ne0', 'gcjsxw3ztvtq', 'gcjvg7d7bx28', 'gcjsmt660upu', 
                     'gcjwwswdgrft', 'gcjm962ms3dr', 'gcmvg8328xz2', 'gcmyt0xj5rzf']

# Remove rows where 'site_id' contains any of the strings in the list

df_wales_cleaner = df_wales[~df_wales['site_id'].str.contains('|'.join(strings_to_remove))]

In [25]:
df_wales_cleaner

,site_id,brand,address,postcode,location.latitude,location.longitude,prices.B7,prices.E10,prices.E5,prices.SDV,Wales
17,gcmg8dcg0n1p,AppleGreen,"Salop Road, Welshpool",SY21 7ET,52.662139,-3.139685,147.8,140.8,NaN,NaN,True
24,gcjmnb068hbn,AppleGreen,"Tremains Road, Bridgend",CF31 1TZ,51.504362,-3.570213,143.8,140.8,NaN,NaN,True
29,gcjw75mdkp2p,AppleGreen,"3 - 3a Lower High Street, Merthyr Tydfil",CF47 8EB,51.741912,-3.376064,151.8,145.8,NaN,NaN,True
40,gcjjyucqgp8d,AppleGreen,"Neath Road, Swansea",SA6 8EF,51.663034,-3.920273,146.8,140.8,155.8,NaN,True
41,gcjjv3exz521,AppleGreen,"1093 Carmarthen Road, Swansea",SA5 4AN,51.64533,-3.98293,NaN,140.8,155.8,NaN,True
42,gcjjxj3ucqjf,AppleGreen,"Fabian Way, Swansea",SA1 8PA,51.621484,-3.908668,147.8,NaN,NaN,162.8,True
43,gcjm3zzvxzsn,AppleGreen,"Port Talbot Road, Swansea",SA13 1HN,51.591409,-3.779302,146.8,141.8,NaN,NaN,True
54,gcmy3c6d7svb,AppleGreen,"Wrexham Road, Mold",CH7 4HL,53.137161,-3.083567,149.8,144.8,NaN,NaN,True
63,gcmy2x6kjhm3,AppleGreen,"Roundabout Service Station King Street, Clwyd",CH7 1LB,53.170418,-3.138778,149.8,144.8,NaN,NaN,True
96,gcjyg47qu66q,BP,"Brecon Road, Brecon Road, Abergavenny.",NP7 7RB,51.825054,-3.027577,147.9,141.9,NaN,NaN,True


In [26]:
len(df_wales_cleaner)

225

In [27]:
#suspicion is some Morrisons duplicates still exist - so filter on Morrisons using .str.contains

df_wales_cleaner[df_wales_cleaner['brand'].str.contains("Morrisons")]

,site_id,brand,address,postcode,location.latitude,location.longitude,prices.B7,prices.E10,prices.E5,prices.SDV,Wales
1409,gcjtqv37tkgy,Morrisons,Castle Court,CF83 1XP,51.577318,-3.217047,146.7,142.7,155.9,NaN,True
1418,gcmwd1d3s9qj,Morrisons,Smithfield Road,LL16 3RQ,53.182333,-3.424443,149.7,146.7,156.7,NaN,True
1421,gcjmcmtbvkpx,Morrisons,Angel Street,SA11 3LN,51.666106,-3.804124,147.7,141.7,NaN,NaN,True
1474,gcjv9bypjmyh,Morrisons,Azalea Road,NP10 9HZ,51.59715,-3.078685,145.7,141.7,NaN,NaN,True
1527,gcjwn92dh306,Morrisons,Bargoed Gateway,CF81 8NX,51.686903,-3.229111,148.7,144.7,155.9,NaN,True
2499,gcjv9bvwc96h,Morrisons,"MFG Morrisons Newport Rogerstone, Azalea Road,...",NP10 9SA,51.597083,-3.079538,145.7,141.7,NaN,NaN,True
2506,gcjwwsks83qs,Morrisons,"MFG Morrisons Ebbw Vale, Beaufort Road, EBBW VALE",NP23 5WS,51.791704,-3.223781,147.7,142.7,NaN,NaN,True
2537,gcmx2d57f5ey,Morrisons,"MFG Morrisons Rhyl, Marsh Road, RHYL",LL18 2DF,53.317314,-3.489098,149.7,145.7,155.9,NaN,True
2538,gcjjy957fcze,Morrisons,"MFG Morrisons Swansea, Morfa Retail Park, SWANSEA",SA1 7DF,51.641889,-3.928514,145.7,139.7,NaN,NaN,True
2568,gcjvg7dmnss1,Morrisons,"MFG Morrisons Cwmbran, Grange Road, CWMBRAN",NP44 1QP,51.655851,-3.017866,147.7,141.7,155.9,NaN,True


In [20]:
#CODE SUPERSEDED - USE THE site_id reference as above

# a number of duplicates still remain - remove again using the .drop method again

#df_wales_cleaner = df_wales_cleaner.drop(index = [1409, 1418, 1421, 1474])

In [30]:
# manually check the above dataframe for Morrisons - see if any duplicates remain - and remove using the .str.contains method again

new_strings_to_remove = ['gcjtqv37tkgy', 'gcmwd1d3s9qj', 'gcjv9bypjmyh']

# Remove rows where 'site_id' contains any of the strings in the list

df_wales_much_cleaner = df_wales_cleaner[~df_wales_cleaner['site_id'].str.contains('|'.join(new_strings_to_remove))]

#### We now have a clean and validated DataFrame of all Wales only fuel stations reporting to the UK Access Fuel site

In [31]:
# returns the final number of rows of the dataframe

len(df_wales_much_cleaner)

222

#### One final bit of housekeeping - in case we want to use the Latitude and Longitdue locations later
**Rename them first - and then we want to change the figures from a string object to a real float number**

In [32]:
#rename lat and longitude columns to single words

df_wales_lat = df_wales_much_cleaner.rename(columns={'location.latitude' : 'latitude', 'location.longitude' : 'longitude'})

In [33]:
#convert lat and long from string object to float numbers

df_wales_lat['latitude'] = df_wales_lat.latitude.astype(float)
df_wales_lat['longitude'] = df_wales_lat.longitude.astype(float)

#### Exporting the final DataFrame to an Excel file format - in this case stored on a hard-drive volume

In [34]:
# export to file

# use datetime to append date to file name


current_date = datetime.now().strftime('%d-%m-%Y')

output = f'fuel_prices_clean_{current_date}.xlsx'

In [35]:
# Specify the path to save the file 
path_to_save = os.path.expanduser('/Volumes/wd_passport/DATA')

# Full file path
file_path = os.path.join(path_to_save, output)

In [36]:
#convert fuel DataFrame data to Excel format and store in directory specified above with date appended

df_wales_lat.to_excel(file_path, index = False)